In [1]:
!pip install pandas

In [2]:
!pip install spacy

In [4]:
!pip install requests

In [6]:
import pandas as pd
import spacy

df = pd.read_excel("answer.xlsx")

result_df = df[['Value']]

result_list = result_df['Value'].tolist()

result = []
for i in range(len(result_list)):
    result.append(str(result_list[i]).split(", "))

In [7]:
import json
with open('test.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

In [9]:
def mapping(ans, relation):
    mapping_list = []
    for i in range(len(ans)):
        if ans[i] == "nan":
            break
        elif relation == "seriesHasNumberOfEpisodes":
            mapping_list.append(ans[i])
        else:
            answer = ans[i]
            if ": " in answer:
                answer_split = answer.split(": ")
                for item in answer_split:

                    entity_wikidata = search_entity_wikidata(item)
                        
                    if entity_wikidata != None:
                        instance = get_instance_of(entity_wikidata['QID'])
                        entity_wikidata['Instance of'] = instance
                        
                        if entity_wikidata['QID'] not in mapping_list:
                            if relation == "awardWonBy":
                                if "human" in entity_wikidata['Instance of']:
                                    mapping_list.append(entity_wikidata['QID'])
                            elif relation == "companyTradesAtStockExchange":
                                if "stock exchange" in entity_wikidata['Instance of']:
                                    mapping_list.append(entity_wikidata['QID'])
                            elif relation == "countryLandBordersCountry":
                                if "country" in entity_wikidata['Instance of']:
                                    mapping_list.append(entity_wikidata['QID'])
                            else: # personHasCityOfDeath
                                for i in range(len(entity_wikidata['Instance of'])):
                                    if "city" in entity_wikidata['Instance of'][i]:
                                        mapping_list.append(entity_wikidata['QID'])
                                        break
            else:
  
                entity_wikidata = search_entity_wikidata(answer)
    
                if entity_wikidata != None:
                    instance = get_instance_of(entity_wikidata['QID'])
                    entity_wikidata['Instance of'] = instance
                    if entity_wikidata['QID'] not in mapping_list:
                        if relation == "awardWonBy":
                            if "human" in entity_wikidata['Instance of']:
                                mapping_list.append(entity_wikidata['QID'])
                        elif relation == "companyTradesAtStockExchange":
                            if "stock exchange" in entity_wikidata['Instance of']:
                                mapping_list.append(entity_wikidata['QID'])
                        elif relation == "countryLandBordersCountry":
                            if "country" in entity_wikidata['Instance of']:
                                mapping_list.append(entity_wikidata['QID'])
                        else: # personHasCityOfDeath
                            for i in range(len(entity_wikidata['Instance of'])):
                                if "city" in entity_wikidata['Instance of'][i]:
                                    mapping_list.append(entity_wikidata['QID'])
                                    break

    return mapping_list
                    

In [10]:
import requests
import time

def search_entity_wikidata(entity_name, language='en'):
    url = 'https://www.wikidata.org/w/api.php'
    params = {
        'action': 'wbsearchentities',
        'search': entity_name,
        'language': language,
        'format': 'json'
    }
    
    response = requests.get(url, params=params)
        
    data = response.json()
    # Check if there are any search results
    if data['search']:
        # Return only QID and Entity name
        results = [{'QID': result['id'], 'EntityName': result['label']} for result in data['search']]
        # Always choose the 1st one
        return results[0]
        
    else: return None

In [11]:
def get_instance_of(qid):
    query = f"""
    SELECT ?instanceOfLabel WHERE {{
      wd:{qid} wdt:P31 ?instanceOf .
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """
    
    url = 'https://query.wikidata.org/sparql'
    headers = {
        'Accept': 'application/sparql-results+json',
    }
    
    response = requests.get(url, params={'query': query}, headers=headers)
    
    if response.status_code == 200:
        results = response.json()['results']['bindings']
        return [result['instanceOfLabel']['value'] for result in results]
    elif response.status_code == 429:
        time.sleep(10)  
        return get_instance_of(qid)
    else:
        response.raise_for_status()

In [12]:
Result_mapping = []

for i in range(len(data)):
    Result_mapping.append(mapping(result[i], data[i]['Relation']))

In [13]:
# Saving Result

df = pd.DataFrame({'Result': [str(lst) for lst in Result_mapping]})

df.to_excel('mapping.xlsx', index=False)

In [14]:
predictions = []
for i in range(len(data)):
    list = {}
    list["SubjectEntity"] = data[i]["SubjectEntity"]
    list["Relation"] = data[i]["Relation"]
    list["ObjectEntitiesID"] = Result_mapping[i]
    predictions.append(list)

In [15]:
fp = "predictions.jsonl"

with open(fp, "w") as f:
    for pred in predictions:
        f.write(json.dumps(pred) + "\n")